## Challenge 03: Minimum Makespan with Quantum Annealing

### Introduction

The core of this challenge is about translating a real-world problem into a mathematical representation that fits a quantum computer. In this challenge we encourage you to use two representations which even though equivalent, are used in different contexts. 

The first is Quadratic Unconstrained Binary Optimization (QUBO) which is used primarily for the quantum annealers (D-Wave) and the other is Ising Hamiltonian, which is often used with an algorithm called Quantum Approximate Optimization Algorithm (QAOA).

This challenge might require spending some time going through the references (especially 1-3), as we are not aware of any "quick introductions" worth mentioning. If you have a hard time understanding something, feel free to reach out for help on [QOSF Slack](https://join.slack.com/t/qosf/shared_invite/zt-bw59w8b9-WJ~k0~FAMHukTZov4AnLfA).

Unless you have a prior background with these kinds of problems, we encourage you to start from QUBO and then perhaps trying Ising Hamiltonians as well, as there are more materials about it.

It is also worth mentioning that D-Wave allows you to use their machines for free for a limited amount of time, so you can try running your QUBO on a real machine after signing up [here](https://cloud.dwavesys.com/leap/signup/).

### Problem

Niranjan owns a catering service and receives orders one day in advance. Each of the chefs he employs is paid by the hour. In order to optimize the cost of running his business, every night, he looks at all of the dishes he needs to deliver for the next day and asks his employees to work the specific hours he requires them for.

Given an order queue ($Q$) of length $N$ with each order $i$ taking $L_i$ time to prepare, if there are $m$ chefs, design a QUBO/Ising Hamiltonian to help Niranjan find the shortest time ($T$) it would take for the $m$ chefs to prepare all $N$ dishes. Assume each chef can only work on one dish at a time and each dish is worked on by only one chef. We can also safely assume that at any given point, all chefs can be actively working on a dish. 

Since Niranjan wants to ensure that he can run his algorithms on the near term devices, keep track and try to minimize the number of:
1. Gates & Qubits (for Ising Hamiltonian)
2. Variables (for QUBO. For embedding on a real machine also keep track of the lenghts of chains)

**Optional:** 
For the advanced reader, you can also attempt to build the Cost Hamiltonian or Mixer Hamiltonian as described in [7,8].

### Examples

#### Case #1: 

*If,*

$N = 5$

$m = 1$

Order Queue = {2, 3, 7, 2, 1}

*then,*

$T = 15$

*since* there is only one chef, the shortest time it would take to complete all the orders is simply the sum of the times it would take for each individual order.

#### Case #2:

*If,*

$N = 5$

$m = 2$

Order Queue = {2, 3, 7, 2, 1}

*then,*

$T = 8$

*since* there are two chefs $m_1$ and $m_2$, the distribution that yields the shortest time is:

$m_1 = \{2, 3, 2\}  $

$m_2 = \{ 7, 1 \} $


#### Case #3:

*If,*

$N = 5$

$m = 3$

Order Queue = {2, 3, 2, 2, 1}

*then,*

$T = 4$

*since* there are three chefs $m_1$, $m_2$ and $m_3$, the distribution that yields the shortest time is:

$m_1 = \{3 \} $

$m_2 = \{2,2 \} $ 

$m_3 = \{2,1 \} $ 

Note that there could be another possible distribution of tasks, but the shortest time does not change:

$m_1 = \{3,1 \} $ 

$m_2 = \{2,2 \} $ 

$m_3 = \{2 \} $

## My Solution

By: [Shiro-Raven](https://github.com/Shiro-Raven)

### Preliminaries
While going through the literature, I came across the [number partitioning problem](https://en.wikipedia.org/wiki/Partition_problem), which aims at dividing a set, $S = \{s_1, s_2, s_3, \cdots, s_m \}$, of numbers into two sets such that the difference between the summation of the numbers of both sets is minimised. In a QUBO context, this translates to having a binary variable, $x_j$, which is equal to one if $s_j$ belongs to subset 1, and 0 otherwise. We then have the following function minimise:
$$
diff^2 = \left(\sum_{j = 1}^{m} s_j - 2\sum_{j = 1}^{m} s_j x_j\right)^2
$$

A general form of this problem, the [multiway number partitioning problem](https://en.wikipedia.org/wiki/Multiway_number_partitioning), attempts to divide the set into $n > 2$ subsets, such that the pairwise difference between the subsets is minimised. We therefore use a binary variable, $x_{ij}$, if $s_i$ belongs to subset $j$. Thus, the function to minimse becomes:
$$
sum_j = \sum_{i = 1}^{n} s_{i}x_{ij}
$$

$$
global\_diff = (sum_1 - sum_2)^2 + (sum_1 - sum_3)^2 + \cdots + (sum_{m - 1} - sum_m)^2
$$

This function needs to be constrained so that no number is assigned to more than one subset, so we have the following constraints:
$$
\sum_{j = 1}^{n} x_{ij} = 1 \;\;\;\; \forall i = 1, \cdots, m
$$

More on the QUBO formulations of these problems can be found [here](http://leeds-faculty.colorado.edu/glover/fred%20pubs/346%20-%20xQx%20-%20Number%20Partitioning%202005.pdf).

### The Approach
The multiway number partitioning and our problem are very similar, by perceiving the subsets as our chefs, and the set of the numbers as our orders. We, however, do not require that the pairwise difference between the sets is minimised. It is sufficient that the pairwise difference between $n - 1$ sets and some predefined set $k$ is minimised. Our solution, $T$, would then be, $sum_k$. This greatly simplifies our QUBO formulation.

Based on this theoritical claim, I used the ```docplex``` package from ```Qiskit``` to generate the QUBO model of the input. I then used the native ```QuadraticProgram```class to transorm this model to a QAOA-compatible form. 

In [1]:
%matplotlib inline
from qiskit.optimization.applications.ising import docplex
from qiskit.aqua.algorithms import NumPyMinimumEigensolver
from qiskit.optimization.algorithms import MinimumEigenOptimizer, RecursiveMinimumEigenOptimizer
from qiskit.optimization import QuadraticProgram
from qiskit.aqua import QuantumInstance

from qiskit import BasicAer
from qiskit.aqua.algorithms import QAOA

import numpy as np

This function takes the inputs and generates the mathematical model as detailed in the reference above.

In [2]:
def generate_qubo(orders_list, nchefs):
    N = len(orders_list)
    C = nchefs
    
    # Create the quadratic model instance
    model = docplex.Model(name='num_part')
    
    # Create the required variables
    # For a problem with N orders and M chefs, we need N*M variables
    x = {(i,j): model.binary_var(name='x_{0}_{1}'.format(i,j)) for i in range(N)
               for j in range(C)}

    sums = [0] * C
    constraints = [0] * N
    for j in range(C):
        for i in range(N):
            sums[j] += orders_list[i] * x[i, j]
            constraints[i] += x[i, j]

    # Object function
    sum_func = model.sum((sums[0] - sums[j])*(sums[0] - sums[j]) for j in range(C))
    
    for i in range(N):
        model.add_constraint(constraints[i] == 1)
        
    model.minimize(sum_func)
        
    qubo = QuadraticProgram()
    qubo.from_docplex(model)

    return qubo

This function takes the problem instances and solves it

In [3]:
def solve(solver, qubo):
    return solver.solve(qubo)

This function analyses the output of the algorithms and prints a human-friendly output

In [4]:
def analyse_solution(sol_masks, orders_list, nchefs):
    masks = sol_masks.reshape(-1, nchefs)
    
    orders_list = np.array(orders_list)
    
    sums = orders_list @ masks
    
    values = (orders_list[:, np.newaxis] * masks).T
    
    for i in range(nchefs):
        orders = np.nonzero(values[i])[0]
        
        print('Chef {} did orders {}, with total duration {}'.format(i, orders, sums[i]))
        
    print('T = {}'.format(np.max(sums)))

We then initialise the instances of the optimisers

In [5]:
quantum_instance = QuantumInstance(BasicAer.get_backend('qasm_simulator'), shots=2000)
qaoa_mes = QAOA(quantum_instance=quantum_instance)
exact_mes = NumPyMinimumEigensolver()

qaoa = MinimumEigenOptimizer(qaoa_mes)   # using QAOA
exact = MinimumEigenOptimizer(exact_mes)  # using the exact classical numpy minimum eigen solver

### Testing

#### Case #1

In [6]:
s = [2, 3, 7, 2, 1]
C = 1

qubo_model = generate_qubo(s, C)

In [7]:
# Classical 
sol = solve(exact, qubo_model)

analyse_solution(sol.x, s, C)

Chef 0 did orders [0 1 2 3 4], with total duration 15.0
T = 15.0


In [8]:
# Quantum
sol = solve(qaoa, qubo_model)

analyse_solution(sol.x, s, C)

Chef 0 did orders [0 1 2 3 4], with total duration 15.0
T = 15.0


#### Case #2

In [9]:
s = [2, 3, 7, 2, 1]
C = 2

qubo_model = generate_qubo(s, C)

In [10]:
# Classical 
sol = solve(exact, qubo_model)

analyse_solution(sol.x, s, C)

Chef 0 did orders [2], with total duration 7.0
Chef 1 did orders [0 1 3 4], with total duration 8.0
T = 8.0


In [11]:
# Quantum
sol = solve(qaoa, qubo_model)

analyse_solution(sol.x, s, C)

Chef 0 did orders [2 4], with total duration 8.0
Chef 1 did orders [0 1 3], with total duration 7.0
T = 8.0


#### Case #3

In [12]:
s = [2, 3, 2, 2, 1]
C = 3

qubo_model = generate_qubo(s, C)

In [13]:
# Classical 
sol = solve(exact, qubo_model)

analyse_solution(sol.x, s, C)

Chef 0 did orders [1], with total duration 3.0
Chef 1 did orders [0 2], with total duration 4.0
Chef 2 did orders [3 4], with total duration 3.0
T = 4.0


In [14]:
# Quantum
sol = solve(qaoa, qubo_model)

analyse_solution(sol.x, s, C)

Chef 0 did orders [0 3], with total duration 4.0
Chef 1 did orders [2 4], with total duration 3.0
Chef 2 did orders [1], with total duration 3.0
T = 4.0


### Problems with the previous implementation

Being a general-purpose quantum programming language, Qiskit can not solve larger instances of the problem. For example, the example below will throw an exception, since it requires 75 qubits. This is way more than what the ```qasm_simulator``` offers (only 24 qubits).

D-Wave's quantum computers can be used to handle problems this large.

#### Case #4

In [15]:
s = [15, 3, 7, 11, 7, 5, 21, 9, 13, 7, 5, 15, 23, 14, 13, 13, 27, 15, 9, 9, 17, 10, 11, 19, 8]
C = 3

### Resources

\[1\] [D-Wave Problem Solving Handbook](https://docs.dwavesys.com/docs/latest/doc_handbook.html)

\[2\] [Implementation of the Travelling Salesman Problem](https://github.com/mstechly/quantum_tsp_tutorials)

\[3\] [Ising formulations of many NP problems](https://arxiv.org/pdf/1302.5843.pdf)

\[4\] [D-Wave Examples](https://docs.ocean.dwavesys.com/en/stable/getting_started.html#examples)

\[5\] [Quantum Approximate Optimization Algorithm explained](https://www.mustythoughts.com/quantum-approximate-optimization-algorithm-explained)

\[6\] [Job Shop Scheduling Solver based on Quantum Annealing](https://arxiv.org/pdf/1506.08479.pdf)

### References

\[7\] [Quantum Algorithms for Scientific Computing and Approximate Optimization](https://arxiv.org/pdf/1805.03265.pdf)

\[8\] [From the Quantum Approximate Optimization Algorithm to a Quantum Alternating Operator Ansatz](https://arxiv.org/pdf/1709.03489.pdf)